In [ ]:
##### Good job!

#OAuth Exercise

In this exercise we will try to scrape twitter data and do a tf-idf analysis on that (src-uwes twitter analysis). We will need OAuth authentication, and we will follow a similar approach as detailed in the yelp analysis notebook. 

In [1]:
import oauth2 as oauth
import urllib

We will now need twitter api access. The following steps as available online will help you set up your twitter account and access the live 1% stream.

1. Create a twitter account if you do not already have one.
2. Go to https://dev.twitter.com/apps and log in with your twitter credentials.
3. Click "Create New App"
4. Fill out the form and agree to the terms. Put in a dummy website if you don't have one you want to use.
5. On the next page, click the "API Keys" tab along the top, then scroll all the way down until you see the section "Your Access Token"
6. Click the button "Create My Access Token". You can Read more about Oauth authorization online. 

Save the details of api_key, api_secret, access_token_key, access_token_secret in your vaule directory and load it in the notebook as shown in yelpSample notebook.

In [2]:
# We need to define the following variables
#api_key = #<get api key> 
#api_secret = #<get api secret>
#access_token_key = #<get your access token key here>"
#access_token_secret = #<get your access token secret here>

#defining them right here is not safe. insteadm create a file in a different directory
# (I use ~/VaultDSE) and in it put a file called, say, twitterkeys.py whose
# content is:
#api_key = #<get api key>
#api_secret = #<get api secret>
#access_token_key = #<get your access token key here>"
#access_token_secret = #<get your access token secret here>
#
#def getkeys():
#    return api_key,api_secret,access_token_key,access_token_secret

# then use the following commands

import sys
sys.path.append('/home/raulmartinez/Documents/VaultDSE') # write your own path here
import TwitterKeys
api_key,api_secret,access_token_key,access_token_secret=TwitterKeys.getkeys()

#api_key,api_secret,access_token_key,access_token_secret

In [3]:
#workaround for SSL Verification

import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

In [4]:
_debug = 0

oauth_token    = oauth.Token(key=access_token_key, secret=access_token_secret)
oauth_consumer = oauth.Consumer(key=api_key, secret=api_secret)

signature_method_hmac_sha1 = oauth.SignatureMethod_HMAC_SHA1()

http_method = "GET"

http_handler  = urllib.request.HTTPHandler(debuglevel=_debug)
https_handler = urllib.request.HTTPSHandler(debuglevel=_debug)

Below is a twitter request method which will use the above user logins to sign, and open a twitter stream request

In [118]:
def getTwitterStream(url, method, parameters):
    req = oauth.Request.from_consumer_and_token(oauth_consumer,
                                             token=oauth_token,
                                             http_method=http_method,
                                             http_url=url, 
                                             parameters=parameters)

    req.sign_request(signature_method_hmac_sha1, oauth_consumer, oauth_token)

    headers = req.to_header()

    if http_method == "POST":
        encoded_post_data = req.to_postdata()
    else:
        encoded_post_data = None
        url = req.to_url()

    opener = urllib.request.OpenerDirector()
    opener.add_handler(http_handler)
    opener.add_handler(https_handler)

    response = opener.open(url, encoded_post_data)

    return response

We can use the above function to request a response as follows

In [6]:
url = "https://stream.twitter.com/1.1/statuses/sample.json"
parameters = []
response = getTwitterStream(url, "GET", parameters)

In [7]:
#print the first three responses
count = 0
for line in response:
    print(line)
    print('\n')
    count+=1
    if count == 3:
        break

b'{"delete":{"status":{"id":1198242897365127168,"id_str":"1198242897365127168","user_id":147534903,"user_id_str":"147534903"},"timestamp_ms":"1575255708231"}}\r\n'


b'{"delete":{"status":{"id":187608948822638593,"id_str":"187608948822638593","user_id":135626797,"user_id_str":"135626797"},"timestamp_ms":"1575255708858"}}\r\n'


b'{"delete":{"status":{"id":655458708751826944,"id_str":"655458708751826944","user_id":2511682935,"user_id_str":"2511682935"},"timestamp_ms":"1575255709511"}}\r\n'




In [8]:
#Now we will test the above function for a sample data provided by twitter stream here -  
url = "https://stream.twitter.com/1.1/statuses/sample.json"

Write a function which will take a url and return the top 10 lines returned by the twitter stream

** Note ** The response returned needs to be intelligently parsed to get the text data which correspond to actual tweets. This part can be done in a number of ways and you are encouraged to try different approaches to parse the response data.

In [15]:
import json
import numpy as np
import copy

def find_key_value(key, dict_):
    """
    Function taken from the website below:
    https://stackoverflow.com/questions/9807634/find-all-occurrences-of-a-key-in-nested-python-dictionaries-and-lists
    
    Not used below, I just took it for testing purposes
    """
    for k, v in (dict_.items() if isinstance(dict_, dict) else
               enumerate(dict_) if isinstance(dict_, list) else []):
        if k == key:
            yield v
        elif isinstance(v, (dict, list)):
            for result in find_key_value(key, v):
                yield result

def fetchData(url, key, top_lines):
    
    #get twitter's response
    parameters = []
    response = getTwitterStream(url, "GET", parameters)
    
    #save top lines
    response_top = []
    i = 0
    for line in response:
        
        try:
            #convert bytest to dictionary using json library
            line_tweet_text = json.loads(line.decode('utf-8'))['text']
            #find key value on multi-level dictionary, use next because function returns a generator (not used)
#             response_top.append(next(find_key_value(key,line_tweet_text))) 
            response_top.append(line_tweet_text) 

        except KeyError:
            response_top.append(np.nan)
        
        #break when top_lines value are found
        i+=1
        if i == top_lines: break
    
    #return response
    return response_top


In [10]:
#Test fetchData function for stream data
response_tweet = fetchData(url = "https://stream.twitter.com/1.1/statuses/sample.json", key = 'text', top_lines=10)
response_tweet

[nan,
 nan,
 nan,
 'Naalala ko bigla wala pala ako utakkk hhshshsh gsnqeqrgyg',
 '귀엽군',
 '@Yes4G Hi,hahaha napee😂',
 '君が君らしく居てくれた時に 僕は僕らしく居られたかなぁ？',
 'RT @Strangestone: 月曜日のたわわ\u3000その２５０ 『愛猫写真フォルダ』 https://t.co/vSUFrzDcx5',
 'RT @_honeyjimin: แจก 100 บาท\nเนื่องจากครอบครัวได้ 8 รางวัลเมื่อวาน\n*เฉพาะอาร์มี่ที่ฟอลโลเวอร์ \n\nรี+ติดแท็ก\n#MAMAVOTE #bts @BTS_twt \n\nประกาศ…',
 '出)レベルボール入りヒトツキ\n     孵化余りミミッキュ\n     孵化余りダルマッカ\n     他ソード限定ポケモン\n\n求)夢サニーゴ\n\n#ポケモン剣盾交換 \n#ポケモン交換']

In [12]:
#We can also request twitter stream data for specific search parameters as follows
search_query = 'example'
url= "https://stream.twitter.com/1.1/statuses/filter.json?track="+search_query #changed to stream API

Call the fetchData function to fetch latest live stream data for following search queries and output the first 5 lines

1. "UCSD"
2. "Donald Trump"
3. "Syria"

In [21]:
#First 5 lines for "UCSD"
search_query = 'UCSD'
response_tweet = fetchData("https://stream.twitter.com/1.1/statuses/filter.json?track="+search_query, key = 'text', top_lines=5)
response_tweet

In [16]:
#First 5 lines for "Donald Trump"
search_query = 'Donald Trump'
response_tweet = fetchData("https://stream.twitter.com/1.1/statuses/filter.json?track="+search_query, key = 'text', top_lines=5)
response_tweet

['RT @Otto_English: "If all of us were caught out ... on a night out after the drink"\n\nNigel Farage defending Donald Trump while raising some…',
 '🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣',
 'RT @charliekirk11: Facts:\n\nJoe Biden held a rally in Crawford County, Iowa—where 67% voted for Trump in 2016\n\n125 people showed up\n\nDonald…',
 'RT @Timcast: News Media is collapsing, salaries are slashed, thousands are laid off, ragebait runs rampant in a desperate bid to bring in m…',
 'RT @SpockResists: If Donald Trump is not impeached and manages to cheat to a second term. \nAmerica is done. His supporters do not understan…']

In [17]:
#First 5 lines for "Syria"
search_query = 'Syria'
response_tweet = fetchData("https://stream.twitter.com/1.1/statuses/filter.json?track="+search_query, key = 'text', top_lines=5)
response_tweet

['RT @evanchill: “Sent candy.” That’s how a Russian pilot confirmed his airstrike on a busy Syrian street this July. Our new investigation re…',
 'RT @shane_bauer: So Halper and @mtaibbi think, “Hey, we should do a show on Syria for Rolling Stone. Who should we have on to explain? The…',
 'RT @minhtngo: #Lebanon: Protesters chant — "revolution in every country" — in solidarity with pro-democracy movements around the world from…',
 'RT @GmanFan45: @SpeakerPelosi your net worth is $195 million. Why are you mooching off my money to get sloshed in Spain??? On a climate hoa…',
 'RT @LoriHandrahan2: Remember #Syria https://t.co/LZ6ap6ZqN7']

In [299]:
#example on how the search query is organized. I just printed this for testing when I was trying to implement it. 
#https://api.twitter.com/1.1/search/tweets.json?q="+search_query
#but I then switched to use a stream track query instread

search_query = 'UCSD'
url = "https://api.twitter.com/1.1/search/tweets.json?q="+search_query
parameters = []
response = getTwitterStream(url, "GET", parameters)

test = [json.loads(i.decode('utf-8')) for i in response]
    
# test.keys()
#print first 5
for i in test[0]['statuses'][:5]:
    print(i)
    print('\n')
# test

{'created_at': 'Sat Nov 30 06:57:01 +0000 2019', 'id': 1200670047212949524, 'id_str': '1200670047212949524', 'text': 'RT @Knit_The_Terror: Reading about the USCGC Maple transiting the Northwest Passage in 2017. Harry Goodsir would be pleased with the zoopla…', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'Knit_The_Terror', 'name': 'Jillian | Mayhem in a High Degree', 'id': 824752702978658304, 'id_str': '824752702978658304', 'indices': [3, 19]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 888996200225157120, 'id_str': '888996200225157120', 'name': 'We Stan Henry Goodsir', 'screen_name': 'Saint_Hemlock', 'location': 'https://ko-fi.com/A7336VR',

### TF-IDF###

tf–idf, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.It is among the most regularly used statistical tool for word cloud analysis. You can read more about it online (https://en.wikipedia.org/wiki/Tf%E2%80%93idf)

We base our analysis on the following

1. The weight of a term that occurs in a document is simply proportional to the term frequency
2. The specificity of a term can be quantified as an inverse function of the number of documents in which it occurs

For this question we will perform tf-idf analysis o the stream data we retrieve for a given search parameter. Perform the steps below

1. use the twitterreq function to search for the query "syria" and save the top 200 lines in the file twitterStream.txt
2. load the saved file and output the count of occurrences for each term. This will be your term frequency
3. Calculate the inverse document frequency for each of the term in the output above.
4. Divide the term frequency for each of the term by corresponding inverse document frequency.
5. Sort the terms in the descending order based on their term freq/inverse document freq scores 
6. Print the top 10 terms.

In [124]:
def twitterreq(url, key, top_lines):
    
    #get twitter's response
    parameters = []
    response = getTwitterStream(url, "GET", parameters)
    
    #save top lines
    response_top = []
    i = 0
    for line in response:
        
        try:
            #convert bytest to dictionary using json library
            line_tweet_text = json.loads(line.decode('utf-8'))['text']
            response_top.append(line_tweet_text) 

        except KeyError:
            response_top.append(np.nan)
        
        #break when top_lines value are found
        i+=1
        if i == top_lines: break
    
    #save tweets as text
    tweets_file = open("twitterStream.txt","w")  
    for tweet_element in response_top:
        tweets_file.write(tweet_element)
        tweets_file.write('\n\n\n\n\n\n\n\n\n\n')
    tweets_file.close() 

In [125]:
#workind directory where file is saved
!pwd

/home/raulmartinez/Documents/DSE/2019-rgm001/DSE200/day_5_mining_the_Social_web/exercises


In [265]:
#1. use the twitterreq function to search for the query "syria" and save the top 200 lines in the file twitterStream.txt

search_query = 'Syria'
twitterreq("https://stream.twitter.com/1.1/statuses/filter.json?track="+search_query, key = 'text', top_lines=200)

In [273]:
#2. load the saved file and output the count of occurrences for each term. This will be your term frequency

#load saved file
tweets_file = open("twitterStream.txt","r") 
inData = tweets_file.readlines()
tweets_file.close() 

#count occurences for each term in each tweet
inData_tweet = ''.join(inData).split('\n\n\n\n\n\n\n\n\n\n')
inData_tweet = inData_tweet[:len(inData_tweet)-1]
frequency = []
tweet_count = 0
for tweet_ in inData_tweet:
    
    #remove certain characters next to words
    list_ = ['(',')','{','}','[',']',',',':','.','?','!','"','-','_','\n','#','/','|']
    for i in list_:
        tweet_ = tweet_.replace(i,'')

    #remove non-ascii characters
    tweet_ = tweet_.encode('ascii',errors='ignore').decode()
    
    #convert every letter to lower-case
    tweet_ = tweet_.lower()
    
    #save all words from tweet as a set, remove empty entries on word list
    word_list = [i for i in list(set(tweet_.split(' '))) if i != '']
    
    #save word and counts as tuples (tweet#,word,word count,word total,term frequency)
    tweet_count+=1  
    for word in word_list:
        frequency.append(('tweet_'+str(tweet_count),word,tweet_.count(word),len(word_list),tweet_.count(word)/len(word_list)))

#convert to dataframe and print head
import pandas as pd
df_frequency = pd.DataFrame(frequency,columns=['tweet_No','word','word count','word total','term frequency'])
df_frequency.head()

,tweet_No,word,word count,word total,term frequency
0,tweet_1,@kurdistannews24,1,15,0.066667
1,tweet_1,rojava,3,15,0.200000
2,tweet_1,riseup4rojava,1,15,0.066667
3,tweet_1,we,1,15,0.066667
4,tweet_1,womendefendrojava,1,15,0.066667


In [274]:
#3. Calculate the inverse document frequency for each of the term in the output above.

"""
idf = obtained by dividing the total number of documents by the number of documents containing 
the term, and then taking the logarithm of that quotient
"""

total_number_documents = len(np.unique(df_frequency.tweet_No))

idf = dict()
for word in np.unique(df_frequency.word):
    
    document_count = 0
    for document in np.unique(df_frequency.tweet_No):
        
        words_list = list(df_frequency.word[df_frequency.tweet_No==document])
        
        if word in words_list:
            document_count+=1
        else:
            continue
    
    idf[word] = document_count    
    
#print dictionary keys
idf.keys()

dict_keys(['$195', '&amp;', "'meat", '1', '1%', '100', '100%', '11', '12', '120', '120km', '14%', '17', '1st', '2', '2010', "2018'deki", '25', '3', '30', '30k', '32', '336', '40', '5', '547', '8', '85', '87', '98%', '@02indian11', '@21wire', '@aaronjmate', '@abarnardnyt', '@afshinrattansi', '@ahvalen', '@ajenglish', '@ambjohnbolton', '@arktinentuuli', '@arnoldisugly', '@astroehlein', '@azadirojava', '@bbcworld', '@bhl', '@borisjohnson', '@canadanato', '@chandershekharj', '@clarkemicah', "@clarkemicah's", '@contesadesange', '@cornishdamo', '@csojourner', '@davidwohl', '@dearauntcrabby', '@desha7', '@dgisserious', '@dknato', '@dlme', '@dublencoigor', '@ekaraduman1070', '@elizabethrajya4', '@emmanuelmacron', '@eucommission', '@euparlement', '@euparliament1', '@eupublicaffairs', '@evanchill', '@evrenwiltse', '@farnazfassihi', '@ferdiknd', '@franceotan', '@freemediahub', '@garzillaman', '@germanydiplo', '@germanynato', '@globeandmail', '@gmanfan45', '@gppi', '@greens', '@gulnuray', '@hevall

In [275]:
#4. Divide the term frequency for each of the term by corresponding inverse document frequency.

tf_idf = []
for word,freq in zip(df_frequency.word,df_frequency['term frequency']):
    tf_idf.append(freq*idf[word])

#add tf_idf to frequency dataframe
df_frequency['tf_idf'] = tf_idf
df_frequency.head()

,tweet_No,word,word count,word total,term frequency,tf_idf
0,tweet_1,@kurdistannews24,1,15,0.066667,0.133333
1,tweet_1,rojava,3,15,0.200000,0.400000
2,tweet_1,riseup4rojava,1,15,0.066667,0.066667
3,tweet_1,we,1,15,0.066667,0.200000
4,tweet_1,womendefendrojava,1,15,0.066667,0.066667


In [276]:
#5 Sort the terms in the descending order based on their term freq/inverse document freq scores 

df_frequency = df_frequency.sort_values(by='tf_idf', ascending=False)
df_frequency

,tweet_No,word,word count,word total,term frequency,tf_idf
17,tweet_2,rt,2,3,0.666667,106.666667
256,tweet_17,rt,2,3,0.666667,106.666667
834,tweet_55,rt,2,5,0.400000,64.000000
1638,tweet_113,rt,2,5,0.400000,64.000000
735,tweet_47,rt,2,5,0.400000,64.000000
...,...,...,...,...,...,...
1391,tweet_96,any,1,26,0.038462,0.038462
1390,tweet_96,footsies,1,26,0.038462,0.038462
1385,tweet_96,eastern,1,26,0.038462,0.038462
2069,tweet_141,want,1,26,0.038462,0.038462


In [277]:
#6 Print the top 10 terms.

#find max value for tf_idf in each word across all tweets (documents)
df_max_tf_idf = df_frequency.groupby(['word'],as_index=False)['tf_idf'].max()

#sort and display top 10
df_max_tf_idf = df_max_tf_idf.sort_values(by='tf_idf',ascending=False) 
df_max_tf_idf = df_max_tf_idf.reset_index(drop=True)
df_max_tf_idf[:10]

,word,tf_idf
0,rt,106.666667
1,syria,32.666667
2,a,32.000000
3,the,24.090909
4,in,20.571429
5,ya,11.000000
6,is,9.900000
7,of,8.250000
8,syrian,7.333333
9,help,5.625000


In [191]:
"""
Example provided but not used for the code
"""

import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "tweet_labelling.settings")
from tweets.models import Tweet
from datetime import datetime
import tweetstream

stream = tweetstream.SampleStream("twitter-id", "twitter-password");
counter=0;
for tweet in stream:
    counter=counter+1;
   
    try:
        print(counter," TEXT: ",tweet["text"])
    except KeyError:
        print("No text field")

    try:
        g=tweet["geo"];
        try:
            if(not g is None):
                print(counter," LOCATION: ",g)
        except AttributeError:
            g=None; #print "AttributeError, g=|",g,"|";
    except KeyError:
        g=None;

ModuleNotFoundError: No module named 'tweets'